# Sharing Processes with a Team

## Connection and Cursor

We will use a connection object to define our connection to our database over a network.

We will use a cursor object to write to the database. The cursor is created by a class function of the connection object.

In [1]:
import psycopg2 as pg2
from psycopg2.extras import RealDictCursor

connection = pg2.connect(host='this_postgis',
                         user='postgres',
                         database='postgres')
cursor = connection.cursor(cursor_factory=RealDictCursor)

The cursor can be used to execute queries and then to fetch the results of the query.

In [2]:
result = cursor.execute("SELECT * FROM business LIMIT 2;")

In [3]:
cursor.fetchall()

[{'id': 10,
  'name': 'TIRAMISU KITCHEN',
  'address': '033 BELDEN PL',
  'city': 'San Francisco',
  'state': 'CA',
  'postal_code': 94104,
  'latitude': 37.791116,
  'longitude': -122.403816,
  'phone_number': '+14154217044'},
 {'id': 19,
  'name': 'NRGIZE LIFESTYLE CAFE',
  'address': '1200 VAN NESS AVE, 3RD FLOOR',
  'city': 'San Francisco',
  'state': 'CA',
  'postal_code': 94109,
  'latitude': 37.786848,
  'longitude': -122.421547,
  'phone_number': '+14157763262'}]

In [4]:
connection.close()

Because we used the `cursor_factory=RealDictCursor` argument, the results return a list of dictionary objects from the database. 

The advantage of this is that we can display these results easily with a DataFrame.


In [5]:
import pandas as pd

In [6]:
connection = pg2.connect(host='this_postgis',
                         user='postgres',
                         database='postgres')
cursor = connection.cursor(cursor_factory=RealDictCursor)
result = cursor.execute("SELECT * FROM business LIMIT 2;")
pd.DataFrame(cursor.fetchall())

,address,city,id,latitude,longitude,name,phone_number,postal_code,state
0,033 BELDEN PL,San Francisco,10,37.791116,-122.403816,TIRAMISU KITCHEN,+14154217044,94104,CA
1,"1200 VAN NESS AVE, 3RD FLOOR",San Francisco,19,37.786848,-122.421547,NRGIZE LIFESTYLE CAFE,+14157763262,94109,CA


In [7]:
connection.close()

## `lib.db_helper`

Because we will keep using the connection-cursor pattern, we have written it into a sub-module.

In [8]:
cd /home/jovyan

/home/jovyan


In [9]:
import lib.db_helper as db

#### `db.connect_to_db()`

Contains our connection credentials and returns a connection and cursor. 

    def connect_to_db():
        con = pg2.connect(host='postgis',
                          dbname='postgres',
                          user='postgres')
        cur = con.cursor(cursor_factory=RealDictCursor)
        return con, cur

In [10]:
connection, cursor = db.connect_to_db()
result = cursor.execute("SELECT * FROM business LIMIT 2;")
pd.DataFrame(cursor.fetchall())

,address,city,id,latitude,longitude,name,phone_number,postal_code,state
0,033 BELDEN PL,San Francisco,10,37.791116,-122.403816,TIRAMISU KITCHEN,+14154217044,94104,CA
1,"1200 VAN NESS AVE, 3RD FLOOR",San Francisco,19,37.786848,-122.421547,NRGIZE LIFESTYLE CAFE,+14157763262,94109,CA


In [11]:
connection.close()

#### `db.query_to_dictionary()`

1. creates a connection and a cursor
1. uses the cursor to execute a query
1. if `fetch_res` is `True` it fetches the results, otherwise results are `None`
1. closes the connection and returns `results`


    def query_to_dictionary(query, fetch_res=True):
        con, cur = connect_to_db()
        cur.execute(query)
        if fetch_res:
            results = cur.fetchall()
        else:
            results = None
        con.close()
        return results

In [12]:
db.query_to_dictionary("""SELECT * FROM business LIMIT 2;""")

[{'id': 10,
  'name': 'TIRAMISU KITCHEN',
  'address': '033 BELDEN PL',
  'city': 'San Francisco',
  'state': 'CA',
  'postal_code': 94104,
  'latitude': 37.791116,
  'longitude': -122.403816,
  'phone_number': '+14154217044'},
 {'id': 19,
  'name': 'NRGIZE LIFESTYLE CAFE',
  'address': '1200 VAN NESS AVE, 3RD FLOOR',
  'city': 'San Francisco',
  'state': 'CA',
  'postal_code': 94109,
  'latitude': 37.786848,
  'longitude': -122.421547,
  'phone_number': '+14157763262'}]

In [13]:
db.query_to_dictionary("""SELECT * FROM business LIMIT 2;""", fetch_res=False)

#### `db.query_to_dataframe()`

Wraps `query_to_dictionary()` in a `pandas.DataFrame`.

    def query_to_dataframe(query):
        return DataFrame(query_to_dictionary(query))

In [14]:
db.query_to_dataframe('SELECT * FROM business LIMIT 2;')

,address,city,id,latitude,longitude,name,phone_number,postal_code,state
0,033 BELDEN PL,San Francisco,10,37.791116,-122.403816,TIRAMISU KITCHEN,+14154217044,94104,CA
1,"1200 VAN NESS AVE, 3RD FLOOR",San Francisco,19,37.786848,-122.421547,NRGIZE LIFESTYLE CAFE,+14157763262,94109,CA


### Create new column in `business` table

To demonstrate how transactions work, we will write a SQL query to create a new column in our database.

#### Run this if the column has already been created and you wish to run all cells in the notebook

In [15]:
# db.query_to_dictionary("""
# BEGIN;
# ALTER TABLE business DROP COLUMN gpnt_location;
# COMMIT;
# """, fetch_res=False)

In [16]:
db.query_to_dictionary("""
BEGIN;
ALTER TABLE business ADD COLUMN gpnt_location geometry(POINT,4326);
COMMIT;
""", fetch_res=False)

In [17]:
db.query_to_dataframe('SELECT * FROM business LIMIT 2;')

,address,city,gpnt_location,id,latitude,longitude,name,phone_number,postal_code,state
0,033 BELDEN PL,San Francisco,None,10,37.791116,-122.403816,TIRAMISU KITCHEN,+14154217044,94104,CA
1,"1200 VAN NESS AVE, 3RD FLOOR",San Francisco,None,19,37.786848,-122.421547,NRGIZE LIFESTYLE CAFE,+14157763262,94109,CA


# Clean the Data

In [18]:
select_count_business = """
SELECT COUNT(*) FROM business
"""
db.query_to_dataframe(select_count_business)

,count
0,6161


In [19]:
select_count_by_postal_code = """
SELECT postal_code, COUNT(*) FROM business
GROUP BY postal_code
ORDER BY postal_code
"""
db.query_to_dataframe(select_count_by_postal_code)

,count,postal_code
0,1,0.0
1,1,84105.0
2,1,92672.0
3,2,94013.0
4,3,94014.0
5,1,94066.0
6,3,94101.0
7,447,94102.0
8,552,94103.0
9,132,94104.0


Note that we might also have invalid data.

In [25]:
select_count_business_invalid_data = """
SELECT COUNT(*) FROM business
WHERE latitude = 0 AND longitude = 0;"""

In [26]:
db.query_to_dataframe(select_count_business_invalid_data)

,count
0,14


In [27]:
select_count_business_valid = """
SELECT COUNT(*) FROM business
WHERE 
    (latitude IS NOT NULL
     AND longitude IS NOT NULL)
AND 
    (latitude != 0 
     AND longitude != 0)
"""

select_count_business_invalid = """
SELECT COUNT(*) FROM business
WHERE 
    (latitude IS NULL
     OR longitude IS NULL)
OR 
    (latitude = 0 
     OR longitude = 0)
"""

In [28]:
db.query_to_dataframe(select_count_business_valid)

,count
0,3735


In [29]:
db.query_to_dataframe(select_count_business_invalid)

,count
0,2426


### Store Valid and Invalid Queries

We will also make use of subqueries. We will define two:

1. a subquery for all businesses with valid lat/long
1. a subquery for all businesses with invalid lat/long

#### Valid Business Query

In [30]:
select_business_valid = """
SELECT * FROM business
WHERE 
    (latitude IS NOT NULL
     AND longitude IS NOT NULL)
AND 
    (latitude != 0 
     AND longitude != 0)
"""

select_business_invalid = """
SELECT * FROM business
WHERE 
    (latitude IS NULL
     OR longitude IS NULL)
OR 
    (latitude = 0 
     OR longitude = 0)
"""

In [31]:
print(db.query_to_dataframe(select_business_valid).shape)
db.query_to_dataframe(select_business_valid).sample(4)

(3735, 10)


,address,city,gpnt_location,id,latitude,longitude,name,phone_number,postal_code,state
3734,475 SACRAMENTO ST,San Francisco,None,71915,37.794063,-122.400726,SAPPHIRE ASIAN CUISINE,+14153781469,94111.0,CA
938,225 11TH ST,San Francisco,None,3393,37.772764,-122.415082,DON RAMON,+14153349141,94103.0,CA
3044,845 HOWARD ST,San Francisco,None,64550,37.782437,-122.403659,CONTINENTAL CAFE & DELI INC.,+14153571060,94103.0,CA
1761,3743 MISSION ST,San Francisco,None,7761,37.736628,-122.424163,LELENITA'S CAKES,+14157598761,94110.0,CA


In [32]:
print(db.query_to_dataframe(select_business_invalid).shape)
db.query_to_dataframe(select_business_invalid).sample(4)

(2426, 10)


,address,city,gpnt_location,id,latitude,longitude,name,phone_number,postal_code,state
1828,3088 BALBOA ST,San Francisco,None,82664,NaN,NaN,PIZZA JOINT,+14153874992,94121.0,CA
2394,314 SUTTER ST,San Francisco,None,86905,NaN,NaN,E & O KITCHEN AND BAR,,94108.0,CA
1211,701 HAIGHT ST,San Francisco,None,79121,NaN,NaN,BLACK SANDS,+14155345194,94117.0,CA
980,2555 JUDAH ST,San Francisco,None,77745,NaN,NaN,TERRA COTTA WARRIOR,+14158067991,94122.0,CA


### Explore Postal Code and Lat/Long

First, let's get the postal codes with a count of less than 10. We will use a sub-query.

Let's use these subqueries to help us to clean the data. 

Note that we re-use `select_count_by_postal_code`.

In [33]:
print(select_count_by_postal_code)


SELECT postal_code, COUNT(*) FROM business
GROUP BY postal_code
ORDER BY postal_code



In [34]:
select_postal_code_by_postal_code_less_than_10 = """
SELECT postal_code FROM 
    ({}) this_subquery
WHERE count < 10
""".format(select_count_by_postal_code)

print(select_postal_code_by_postal_code_less_than_10)


SELECT postal_code FROM 
    (
SELECT postal_code, COUNT(*) FROM business
GROUP BY postal_code
ORDER BY postal_code
) this_subquery
WHERE count < 10



Note that we are using a subquery here. 

e.g. 

    SELECT * FROM (SUBQUERY) sub_query_name;
    

In [35]:
db.query_to_dataframe(select_postal_code_by_postal_code_less_than_10)

,postal_code
0,0
1,84105
2,92672
3,94013
4,94014
5,94066
6,94101
7,94120
8,94129
9,94130


We'll think of these as our "bad postal codes". We can think of the query we defined as a list of these codes. That query was

In [36]:
bad_postal_codes = select_postal_code_by_postal_code_less_than_10

In [37]:
print(bad_postal_codes)


SELECT postal_code FROM 
    (
SELECT postal_code, COUNT(*) FROM business
GROUP BY postal_code
ORDER BY postal_code
) this_subquery
WHERE count < 10



Do we have (lat,long) pairs for these locations? Let's select all rows with one of these postal codes.

### `IN`

To do this we will use the SQL keyword `IN`. `IN` checks a column against a list of items.

e.g. 

    SELECT * FROM my_table WHERE my_column IN ('1,2,3,4,5');

# We will replace the list with our Sub-Query!! 

In [38]:
select_business_where_bad_postal_code = """
SELECT * FROM business WHERE postal_code IN ({})""".format(bad_postal_codes)
print(select_business_where_bad_postal_code)


SELECT * FROM business WHERE postal_code IN (
SELECT postal_code FROM 
    (
SELECT postal_code, COUNT(*) FROM business
GROUP BY postal_code
ORDER BY postal_code
) this_subquery
WHERE count < 10
)


In [39]:
db.query_to_dataframe(select_business_where_bad_postal_code).sample(5)

,address,city,gpnt_location,id,latitude,longitude,name,phone_number,postal_code,state
15,PUBLIC RIGHT OF WAY,San Francisco,None,65673,37.834628,-122.264171,DOC'S OF THE BAY,+14156883245,94609,CA
24,"200 CALIFORNIA AVE., BLDG 180S, TREASURE ISLAND",San Francisco,None,78650,NaN,NaN,TREASURE BRANDS LLC,+14152919192,94130,CA
26,401 13TH ST,San Francisco,None,79829,NaN,NaN,ARACELY RESTAURANT,+14156940363,94130,CA
10,TREASURE ISLAND 888 AVENUE H,San Francisco,None,63091,NaN,NaN,MAYA WATERS,+14156137162,94130,CA
25,OFF THE GRID,San Francisco,None,79607,NaN,NaN,TAQUERIA ANGELICA'S #2,+14158343695,94129,CA


Note that some of these have a lat and long. 

What we want is rows that have "bad postal codes" and no (lat, long) pair. 

### A Nested Query

To get these we will write a nested query that looks like this:

    SELECT * FROM (invalid_lat_long) WHERE postal_code IN (bad_postal_code_str)

In [40]:
select_invalid_business_where_bad_postal_code = """
SELECT * 
FROM (""" + select_business_invalid + """) this_subquery
WHERE postal_code IN (""" + bad_postal_codes + """)
"""
print(select_invalid_business_where_bad_postal_code)


SELECT * 
FROM (
SELECT * FROM business
WHERE 
    (latitude IS NULL
     OR longitude IS NULL)
OR 
    (latitude = 0 
     OR longitude = 0)
) this_subquery
WHERE postal_code IN (
SELECT postal_code FROM 
    (
SELECT postal_code, COUNT(*) FROM business
GROUP BY postal_code
ORDER BY postal_code
) this_subquery
WHERE count < 10
)



In [41]:
select_invalid_business_where_bad_postal_code = """
SELECT * 
FROM ({}) q
WHERE postal_code IN ({})
""".format(select_business_invalid, bad_postal_codes)
print(select_invalid_business_where_bad_postal_code)


SELECT * 
FROM (
SELECT * FROM business
WHERE 
    (latitude IS NULL
     OR longitude IS NULL)
OR 
    (latitude = 0 
     OR longitude = 0)
) q
WHERE postal_code IN (
SELECT postal_code FROM 
    (
SELECT postal_code, COUNT(*) FROM business
GROUP BY postal_code
ORDER BY postal_code
) this_subquery
WHERE count < 10
)



In [42]:
db.query_to_dataframe(select_invalid_business_where_bad_postal_code)

,address,city,gpnt_location,id,latitude,longitude,name,phone_number,postal_code,state
0,VARIOUS LOACATIONS (17),San Francisco,None,5755,None,None,J & J VENDING,+14156750910,94545,CA
1,VARIOUS LOCATIONS,San Francisco,None,5757,None,None,"RICO VENDING, INC",+14155836723,94066,CA
2,390 CLEMENTINA ST,San Francisco,None,6029,None,None,ALEXIS APARTMENTS SR. MEALS PROGRAM,+14154959541,941033148,CA
3,TREASURE ISLAND 888 AVENUE H,San Francisco,None,63091,None,None,MAYA WATERS,+14156137162,94130,CA
4,"150 04TH ST., TREASURE ISLAND SUITE 1",San Francisco,None,69804,None,None,ISLAND COVE MARKET,+14153912299,94130,CA
5,"HUNTER'S POINT SHIPYARD, BLDG.110",San Francisco,None,71471,None,None,O'CACAO,+14159062574,94188,CA
6,5383 CAPWELL,San Francisco,None,72127,None,None,REVOLUTION FOODS,,94621,CA
7,150 04TH ST SUITE 1,San Francisco,None,73799,None,None,ISLAND COVE MARKET,+14153912299,94130,CA
8,101 BAYSHORE BLVD,San Francisco,None,74674,None,None,ELI'S HOT DOGS,+14158301168,94014,CA
9,2769 LOMBARD ST,San Francisco,None,77759,None,None,PIZZERIA AVELLINO,+14157762500,94129,CA


#### Let's use `DELETE` to get rid of rows that have no lat, long and a bad zip

First, let's collect a list of business `id`s. 

In [43]:
select_invalid_business_id_where_bad_postal_code = """
SELECT id 
FROM ({}) q
WHERE postal_code IN ({})
""".format(select_business_invalid, select_postal_code_by_postal_code_less_than_10)
print(select_invalid_business_id_where_bad_postal_code)


SELECT id 
FROM (
SELECT * FROM business
WHERE 
    (latitude IS NULL
     OR longitude IS NULL)
OR 
    (latitude = 0 
     OR longitude = 0)
) q
WHERE postal_code IN (
SELECT postal_code FROM 
    (
SELECT postal_code, COUNT(*) FROM business
GROUP BY postal_code
ORDER BY postal_code
) this_subquery
WHERE count < 10
)



In [44]:
db.query_to_dataframe(select_invalid_business_id_where_bad_postal_code)

,id
0,5755
1,5757
2,6029
3,63091
4,69804
5,71471
6,72127
7,73799
8,74674
9,77759


In [45]:
print(select_invalid_business_id_where_bad_postal_code)


SELECT id 
FROM (
SELECT * FROM business
WHERE 
    (latitude IS NULL
     OR longitude IS NULL)
OR 
    (latitude = 0 
     OR longitude = 0)
) q
WHERE postal_code IN (
SELECT postal_code FROM 
    (
SELECT postal_code, COUNT(*) FROM business
GROUP BY postal_code
ORDER BY postal_code
) this_subquery
WHERE count < 10
)



We can use this subquery in our `DELETE` query.

Remember, that a **delete** action is a write action and needs to be handled as a transaction. We need to `BEGIN` and `COMMIT`. 

In [46]:
delete_invalid_business_bad_postal_code = """
BEGIN;
DELETE 
FROM business
WHERE id IN ({});
COMMIT;
""".format(select_invalid_business_id_where_bad_postal_code)
print(delete_invalid_business_bad_postal_code)


BEGIN;
DELETE 
FROM business
WHERE id IN (
SELECT id 
FROM (
SELECT * FROM business
WHERE 
    (latitude IS NULL
     OR longitude IS NULL)
OR 
    (latitude = 0 
     OR longitude = 0)
) q
WHERE postal_code IN (
SELECT postal_code FROM 
    (
SELECT postal_code, COUNT(*) FROM business
GROUP BY postal_code
ORDER BY postal_code
) this_subquery
WHERE count < 10
)
);
COMMIT;



Remember, when we execute this we do not need to fetch results so we should set `fetch_res` to `False`.

In [47]:
db.query_to_dictionary(delete_invalid_business_bad_postal_code, fetch_res=False)

IntegrityError: update or delete on table "business" violates foreign key constraint "fk_business_id" on table "inspection"
DETAIL:  Key (id)=(5755) is still referenced from table "inspection".


Let's have a look at the sql file used to define our database. 

```SQL
CREATE TABLE business (
    id INTEGER,
    name TEXT,
    address TEXT,
    city TEXT,
    state TEXT,
    postal_code INTEGER,
    latitude FLOAT,
    longitude FLOAT,
    phone_number TEXT,
    PRIMARY KEY (id));

CREATE TABLE inspection (
    business_id INTEGER,
    score INTEGER,
    date TIMESTAMP,
    type TEXT,
    CONSTRAINT fk_business_id
    FOREIGN KEY (business_id)
    REFERENCES business (id));

CREATE TABLE legend (
    minimum_score INTEGER,
    maximum_score INTEGER,
    description TEXT);

CREATE TABLE violation (
    business_id INTEGER,
    date TIMESTAMP,
    description TEXT,
    CONSTRAINT fk_business_id
    FOREIGN KEY (business_id)
    REFERENCES business (id));

```

Note that we have foreign key `CONSTRAINT`s on the `inspection` and `violation` tables.  

This means that, in order to drop the businesses with "bad" data, we will need to drop any inspections and violations associated with these. 

### "Bad" Business Data

In [48]:
db.query_to_dataframe(select_invalid_business_id_where_bad_postal_code)

,id
0,5755
1,5757
2,6029
3,63091
4,69804
5,71471
6,72127
7,73799
8,74674
9,77759


In [49]:
print(select_invalid_business_id_where_bad_postal_code)


SELECT id 
FROM (
SELECT * FROM business
WHERE 
    (latitude IS NULL
     OR longitude IS NULL)
OR 
    (latitude = 0 
     OR longitude = 0)
) q
WHERE postal_code IN (
SELECT postal_code FROM 
    (
SELECT postal_code, COUNT(*) FROM business
GROUP BY postal_code
ORDER BY postal_code
) this_subquery
WHERE count < 10
)



In [50]:
select_violations_for_bad_biz = """
SELECT * FROM violation
WHERE business_id IN ({})
""".format(select_invalid_business_id_where_bad_postal_code)

In [51]:
db.query_to_dataframe(select_violations_for_bad_biz)

,business_id,date,description
0,6029,2014-03-13,Food safety certificate or food handler card n...
1,6029,2014-03-13,Unapproved or unmaintained equipment or utensi...
2,69804,2015-06-17,Low risk vermin infestation
3,69804,2015-06-17,Moderate risk food holding temperature
4,69804,2013-09-26,No thermometers or uncalibrated thermometers ...
5,69804,2013-09-26,Inadequate food safety knowledge or lack of ce...
6,71471,2013-05-02,Improper food storage
7,74674,2015-08-18,Inadequate and inaccessible handwashing facili...
8,74674,2015-08-18,Permit license or inspection report not posted...
9,74674,2015-08-18,Mobile food facility with unapproved operating...


### Delete Violations

In [52]:
select_count_violation = """
SELECT COUNT(*) FROM violation"""

In [53]:
db.query_to_dataframe(select_count_violation)

,count
0,39907


In [54]:
delete_violations_for_bad_biz = """
BEGIN;
DELETE FROM violation
WHERE business_id IN ({});
COMMIT;""".format(select_invalid_business_id_where_bad_postal_code)

In [55]:
db.query_to_dictionary(delete_violations_for_bad_biz, fetch_res=False)

In [56]:
db.query_to_dataframe(select_count_violation)

,count
0,39863


### Delete Inspections

In [57]:
select_count_inspections = """
SELECT COUNT(*) FROM inspection"""

In [58]:
db.query_to_dataframe(select_count_inspections)

,count
0,15429


In [59]:
delete_inspections_for_bad_biz = """
BEGIN;
DELETE FROM inspection
WHERE business_id IN ({});
COMMIT;""".format(select_invalid_business_id_where_bad_postal_code)

In [60]:
db.query_to_dictionary(delete_inspections_for_bad_biz, fetch_res=False)

In [61]:
db.query_to_dataframe(select_count_inspections)

,count
0,15408


### Delete Businesses

In [62]:
db.query_to_dataframe(select_count_business)

,count
0,6161


In [63]:
db.query_to_dictionary(delete_invalid_business_bad_postal_code, fetch_res=False)

In [64]:
db.query_to_dataframe(select_count_business)

,count
0,6138


### Pull Bad Postal Codes Again

In [65]:
db.query_to_dataframe(select_postal_code_by_postal_code_less_than_10)

,postal_code
0,0
1,92672
2,94013
3,94101
4,94120
5,94130
6,94140
7,94143
8,94513
9,94609


## Repair Bad Zip Codes
To repair the bad zip codes, we will attempt match the nearest points using GIS. 

To do this, we will first need to populate the `gpnt_location` column.

In [66]:
db.query_to_dataframe(select_business_valid).sample(4)

,address,city,gpnt_location,id,latitude,longitude,name,phone_number,postal_code,state
770,2877 CALIFORNIA ST,San Francisco,None,2866,37.787884,-122.441657,ELIZA'S RESTAURANT,+14156489999,94115.0,CA
2721,101 SPEAR ST,San Francisco,None,60475,37.792577,-122.393992,ARABI MEDITERRANEAN CUISINE,+14152438575,94105.0,CA
3422,1541 CLEMENT ST,San Francisco,None,68220,37.782383,-122.475929,CHOMP & SWIG,+14155212364,94118.0,CA
1799,1301 FILLMORE STREET,San Francisco,None,9232,37.781974,-122.432474,FILLMORE STREET CAFE,+14157490987,94115.0,CA


In [67]:
update_gpnt = """
BEGIN;
UPDATE business SET 
gpnt_location = ST_SetSRID(ST_MakePoint(latitude, longitude),4326);
COMMIT;
"""

In [68]:
db.query_to_dictionary(update_gpnt, fetch_res=False)

### Match points to the points with bad zip codes using a given radius

In [69]:
db.query_to_dataframe(select_postal_code_by_postal_code_less_than_10)

,postal_code
0,0
1,92672
2,94013
3,94101
4,94120
5,94130
6,94140
7,94143
8,94513
9,94609


In [70]:
def select_postal_code(postal_code):
    return """SELECT * FROM business WHERE postal_code = {}""".format(postal_code)

In [71]:
db.query_to_dataframe(select_postal_code(92672))

,address,city,gpnt_location,id,latitude,longitude,name,phone_number,postal_code,state
0,1530 HAIGHT ST,San Francisco,0101000020E6100000C32973F38DE242405FB87361A49C...,64660,37.769957,-122.447533,HAIGHT STREET MARKET,+14152550643,92672,CA


In [72]:
def select_gpnt_for_postal_code(postal_code):
    return """SELECT gpnt_location FROM business WHERE postal_code = {} LIMIT 1""".format(postal_code)

In [73]:
def select_on_gpnt_radius(postal_code, distance):
    return """
            SELECT *
            FROM business
            WHERE ST_Distance_Sphere(gpnt_location, ({})) <= {}
            """.format(select_gpnt_for_postal_code(postal_code), distance)

In [74]:
db.query_to_dataframe(select_on_gpnt_radius(92672, 100))

,address,city,gpnt_location,id,latitude,longitude,name,phone_number,postal_code,state
0,1476 HAIGHT ST,San Francisco,0101000020E61000001BD9959691E24240193DB7D0959C...,1048,37.770068,-122.446644,CREPE EXPRESS,+14158650264,94117,CA
1,1509 HAIGHT ST,San Francisco,0101000020E6100000B8AF03E78CE24240DFDDCA129D9C...,4257,37.769925,-122.447087,DELUXE,+14155526949,94117,CA
2,1480 HAIGHT ST,San Francisco,0101000020E61000001BD9959691E24240193DB7D0959C...,4896,37.770068,-122.446644,BEN & JERRY'S HAIGHT-ASHBURY,+14154573140,94117,CA
3,1535 HAIGHT ST,San Francisco,0101000020E6100000AD3594DA8BE2424040852348A59C...,36635,37.769893,-122.447588,BIG SLICE PIZZA,+14155522520,94117,CA
4,1530 HAIGHT ST,San Francisco,0101000020E6100000C32973F38DE242405FB87361A49C...,64660,37.769957,-122.447533,HAIGHT STREET MARKET,+14152550643,92672,CA
5,1599 HAIGHT ST,San Francisco,0101000020E61000006DAE9AE788E24240B1FB8EE1B19C...,67587,37.769803,-122.448357,BURGER URGE,+14155220122,94117,CA
6,1568 HAIGHT ST,San Francisco,0101000020E61000009BCAA2B08BE242404224438EAD9C...,70970,37.769888,-122.448093,H.Q. BAR & RESTAURANT,,94117,CA


In [75]:
update_92672 = """
BEGIN;
UPDATE business SET postal_code = 94117 WHERE postal_code = 92672;
COMMIT;
"""

In [76]:
db.query_to_dictionary(update_92672, fetch_res=False)

### Check the Status of our Bad Postal Codes

In [77]:
db.query_to_dataframe(select_postal_code_by_postal_code_less_than_10)

,postal_code
0,0
1,94013
2,94101
3,94120
4,94130
5,94140
6,94143
7,94513
8,94609
9,95105


In [78]:
db.query_to_dataframe(select_on_gpnt_radius(94101, 100))

,address,city,gpnt_location,id,latitude,longitude,name,phone_number,postal_code,state
0,1555 MISSION ST,San Francisco,0101000020E610000082C476F700E34240F06B2409C29A...,1216,37.773467,-122.418093,ZAOH RESTAURANT,+14154313930,94103,CA
1,1455A MARKET ST,San Francisco,0101000020E6100000BEC1172653E34240962023A0C29A...,4661,37.775975,-122.418129,HAPPY DONUT,+14152528868,94103,CA
2,1455 MARKET ST 1/F,San Francisco,0101000020E61000005EB9DE3653E34240DEE68D93C29A...,5018,37.775977,-122.418126,MCDONALDS,+14154686990,94103,CA
3,1559 MISSION ST,San Francisco,0101000020E61000003B18B14F00E3424025ADF886C29A...,36745,37.773447,-122.418123,SHANGHAI CHINA RESTAURANT,+14156218188,94103,CA
4,41 MONTGOMERY ST,San Francisco,0101000020E6100000923B6C2233E34240C53C2B69C59A...,65856,37.774998,-122.418299,BAMBOO ASIA,+14156246790,94101,CA
5,1400 MARKET ST,San Francisco,0101000020E61000006D54A70359E3424008C89750C19A...,66219,37.776154,-122.418049,LITTLE GRIDDLE,+14158644292,94102,CA
6,1408 MARKET ST,San Francisco,0101000020E61000004A7EC4AF58E3424032755776C19A...,69121,37.776144,-122.418058,"MAVELOUS, LLC",+14156268884,94102,CA


## Plot locations

Next, we will use Folium to plot the businesses for which we have a latitude and a longitude. 

### `folium.Map` 

To build our `folium.Map` object we will need to center the map in a given location. We will use the `avg` aggregate function to do this. 

In [79]:
import folium

In [80]:
avg_location_sf = """
SELECT avg(latitude) as avg_lat, 
       avg(longitude) as avg_lon FROM ({}) q;
""".format(select_business_valid)

In [81]:
db.query_to_dataframe(avg_location_sf)

,avg_lat,avg_lon
0,37.774003,-122.425144


In [82]:
avg_loc_df = db.query_to_dataframe(avg_location_sf)

Then assign the result as a single list value.

In [83]:
avg_loc = avg_loc_df.values.tolist()[0]

In [84]:
sf_map = folium.Map(location=avg_loc, zoom_start=13)

In [85]:
within_200_meters_of_94101 = db.query_to_dataframe(select_on_gpnt_radius(94101, 200))

In [86]:
near_94101_lat_long = within_200_meters_of_94101[['latitude', 'longitude','postal_code']].values.tolist()

In [87]:
for loc in near_94101_lat_long:
    try:
        postal_code = str(int(loc[2]))
    except:
        postal_code = None
    folium.Marker(loc[:2],postal_code).add_to(sf_map)

In [88]:
sf_map